In [2]:
import os
import librosa
import numpy as np

#@markdown ### Path to your audio file
audio_path = "/content/Mechanical_Mirage.mp3" #@param {type:"string"}

# Load the audio file
y, sr = librosa.load(audio_path, sr=None)
# Extract tempo and beats
tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
beat_times = librosa.frames_to_time(beats, sr=sr)

# Round beat times to hundredth of a second
beat_times_rounded = [round(time, 2) for time in beat_times]
# Calculate frame numbers at 10 fps based on beat times
frame_numbers_10fps = [int(time * 10) for time in beat_times_rounded]
max_frame = max(frame_numbers_10fps)

# Initialize switch and inverse switch frames
current_state = 0
switch_frames = []
invert_switch_frames = []
for i in range(max_frame + 1):
    if i in frame_numbers_10fps:
        current_state = 1 - current_state  # Toggle between 0 and 1
    switch_frames.append(f"{i + 1}:({current_state})")
    invert_switch_frames.append(f"{i + 1}:({1 - current_state})")

# Save the switch effect frames
output_path_switch = '/content/beattimes_switch.txt'
with open(output_path_switch, 'w') as file:
    file.write(',\n'.join(switch_frames))
print(f"Switch effect frame list saved to: {output_path_switch}")

# Save the inverse switch effect frames
output_path_invert_switch = '/content/invert_beattimes_switch.txt'
with open(output_path_invert_switch, 'w') as file:
    file.write(',\n'.join(invert_switch_frames))
print(f"Inverse switch effect frame list saved to: {output_path_invert_switch}")

# Function to interpolate beat changes with smooth gradient
def interpolate_beat_changes(beat_times, fps=10):
    frame_numbers = [int(time * fps) for time in beat_times]
    beat_wave = np.zeros(max(frame_numbers) + 1)
    for i in range(len(frame_numbers) - 1):
        start_frame = frame_numbers[i]
        end_frame = frame_numbers[i + 1]
        beat_wave[start_frame:end_frame] = np.linspace(0, 1, end_frame - start_frame, endpoint=False)
    return beat_wave

# Generate and save the continuous beat wave
beat_wave = interpolate_beat_changes(beat_times_rounded)
output_path_wave = '/content/beatwaves.txt'
with open(output_path_wave, 'w') as file:
    formatted_data_wave = ',\n'.join([f"{i + 1}:({value:.2f})" for i, value in enumerate(beat_wave)])
    file.write(formatted_data_wave)
print(f"Continuous beat wave data saved to: {output_path_wave}")

# Generate and save the inverse continuous beat wave
inverse_beat_wave = 1 - beat_wave
output_path_inverse_wave = '/content/invert_beatwaves.txt'
with open(output_path_inverse_wave, 'w') as file:
    formatted_data_inverse_wave = ',\n'.join([f"{i + 1}:({value:.2f})" for i, value in enumerate(inverse_beat_wave)])
    file.write(formatted_data_inverse_wave)
print(f"Inverse continuous beat wave data saved to: {output_path_inverse_wave}")

Switch effect frame list saved to: /content/beattimes_switch.txt
Inverse switch effect frame list saved to: /content/invert_beattimes_switch.txt
Continuous beat wave data saved to: /content/beatwaves.txt
Inverse continuous beat wave data saved to: /content/invert_beatwaves.txt
